In [11]:
import torch
import torch.nn as nn
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
import math
import zstandard as zstd
import datetime
from satkit import time
import io

def read_zst(filepath:str):
    lines = []
    with open(filepath, "rb") as file:
        decompressor = zstd.ZstdDecompressor()
        reader = decompressor.stream_reader(file)
        text_stream = io.TextIOWrapper(reader, encoding="utf-8")

        for line in text_stream:
            lines.append(line.rstrip("\n"))
    return lines


class State:
    def __init__(self, line:str):
        orbital_data = line.split(",")

        dt_time = datetime.datetime.fromtimestamp(float(orbital_data[0].rstrip()))
        self.time = time.from_datetime(dt_time)
        self.pos_x = float(orbital_data[1])
        self.pos_y = float(orbital_data[2])
        self.pos_z = float(orbital_data[3])
        self.vel_x = float(orbital_data[4])
        self.vel_y = float(orbital_data[5])
        self.vel_z = float(orbital_data[6])
    
    def get_position_vector(self):
        """
        Returns the position vector as a numpy array
        """
        return np.array([self.pos_x, self.pos_y, self.pos_z])
    
    def get_velocity_vector(self):
        """
        Returns the velocity vector as a numpy array
        """
        return np.array([self.vel_x, self.pos_y, self.pos_z])
    
    def get_velocity_magnitude(self):
        """
        Returns magnitude of the velocity vector, in km/s
        """
        return math.sqrt(self.vel_x**2 + self.vel_y**2 + self.vel_z**2)/1000

In [13]:
FILEPATH = "/mnt/IronWolfPro8TB/SWARM/data/output/raw/integration_12_0.txt.zst"

lines = read_zst(FILEPATH)
states = []
for line in lines:
    s = State(line)
    states.append(s)


for i in range(0, 10):
    print(states[i].time)
    v_magnitude = states[i].get_velocity_magnitude()
    print(f"Magnitude of Velocity Vector: {v_magnitude}")

2023-12-31T18:56:11.167296Z
Magnitude of Velocity Vector: 7999.141716103813
2023-12-31T18:56:24.725998Z
Magnitude of Velocity Vector: 7999.141713407197
2023-12-31T18:56:38.284700Z
Magnitude of Velocity Vector: 7999.141710677208
2023-12-31T18:56:51.843402Z
Magnitude of Velocity Vector: 7999.14170791373
2023-12-31T18:57:05.402104Z
Magnitude of Velocity Vector: 7999.14170511665
2023-12-31T18:57:18.960806Z
Magnitude of Velocity Vector: 7999.141702285854
2023-12-31T18:57:32.519508Z
Magnitude of Velocity Vector: 7999.141699421238
2023-12-31T18:57:46.078210Z
Magnitude of Velocity Vector: 7999.141696522691
2023-12-31T18:57:59.636912Z
Magnitude of Velocity Vector: 7999.141693590119
2023-12-31T18:58:13.195614Z
Magnitude of Velocity Vector: 7999.141690623424


In [14]:
import satkit
from satkit import satstate

#TODO: Implement satkit satstates to begin training from integrated data

#satkit.utils.update_datafiles()

for i in range(0,10):
    dt = states[i].time #datetime object
    p = states[i].get_position_vector()
    v = states[i].get_velocity_vector()

    new_state = satstate(dt, p, v)

In [15]:
import customMLDSGP4

model = customMLDSGP4.mldsgp4()

